<a href="https://colab.research.google.com/github/pratik-poudel/datacrunch/blob/main/feature_selection_and_training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd
from collections import defaultdict
from tqdm.notebook import tqdm
from matplotlib import pyplot as plt
import warnings
warnings.filterwarnings('ignore')
import seaborn as sns
import gc
# pd.options.display.float_format| = "{:.2f}".format
pd.options.display.max_columns = 500


from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score 
from sklearn.metrics import classification_report , mean_squared_error, make_scorer
import requests
from scipy import stats
from numpy import absolute
from numpy import mean
from numpy import std

import lightgbm as lgb

from sklearn.datasets import make_regression
from sklearn.linear_model import LinearRegression
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedKFold
from sklearn.multioutput import RegressorChain
from sklearn.multioutput import MultiOutputRegressor
from sklearn.ensemble import ExtraTreesRegressor
from sklearn.svm import LinearSVR
from sklearn.linear_model import SGDRegressor


In [2]:
train_datalink_X = 'https://tournament.datacrunch.com/data/X_train.csv'  
train_datalink_y = 'https://tournament.datacrunch.com/data/y_train.csv'
hackathon_data_link = 'https://tournament.datacrunch.com/data/X_test.csv'

In [3]:
train = pd.read_csv(train_datalink_X)
target = pd.read_csv(train_datalink_y)
test = pd.read_csv(hackathon_data_link)

In [4]:
def additional_features(df):
    df['feature_17+feature_23+feature_8*feature_21'] = df['feature_17'] + df['feature_23'] + df['feature_8'] * df['feature_21']
    df['feature_23+feature_4+feature_8+feature_21'] = df['feature_23'] + df['feature_4'] + df['feature_8'] + df['feature_21']
    df['feature_18+feature_23+feature_8*feature_21'] = df['feature_18'] + df['feature_23'] + df['feature_8'] * df['feature_21']
    df['feature_23+feature_11+feature_8+feature_21'] = df['feature_23'] + df['feature_11'] + df['feature_8'] + df['feature_21']
    df['feature_17+feature_18+feature_23+feature_21'] = df['feature_17'] + df['feature_18'] + df['feature_23'] + df['feature_21']
    df['feature_23+feature_12+feature_8+feature_21'] = df['feature_23'] + df['feature_12'] + df['feature_8'] + df['feature_21']
    df['feature_17+feature_19+feature_23+feature_21'] = df['feature_17'] + df['feature_19'] + df['feature_23'] + df['feature_21']
    df['feature_23+feature_5+feature_8+feature_21'] = df['feature_23'] + df['feature_5'] + df['feature_8'] + df['feature_21']
    df['feature_23+feature_3+feature_8+feature_21'] = df['feature_23'] + df['feature_3'] + df['feature_8'] + df['feature_21']
    df['feature_18+feature_19+feature_23+feature_21'] = df['feature_18'] + df['feature_19'] + df['feature_23'] + df['feature_21']
    df['feature_23+feature_9+feature_8+feature_21'] = df['feature_23'] + df['feature_9'] + df['feature_8'] + df['feature_21']
    df['feature_19+feature_23+feature_8*feature_21'] = df['feature_19'] + df['feature_23'] + df['feature_8'] * df['feature_21']
    df['feature_17+feature_23+feature_16*feature_21'] = df['feature_17'] + df['feature_23'] + df['feature_16'] * df['feature_21']
    df['feature_17+feature_23+feature_8+feature_21'] = df['feature_17'] + df['feature_23'] + df['feature_8'] + df['feature_21']
    df['feature_18+feature_23+feature_16*feature_21'] = df['feature_18'] + df['feature_23'] + df['feature_16'] * df['feature_21']
    df['feature_17+feature_23+feature_3*feature_21'] = df['feature_17'] + df['feature_23'] + df['feature_3'] * df['feature_21']
    df['feature_17+feature_23+feature_3+feature_21'] = df['feature_17'] + df['feature_23'] + df['feature_3'] + df['feature_21']
    df['feature_18+feature_23+feature_8+feature_21'] = df['feature_18'] + df['feature_23'] + df['feature_8'] + df['feature_21']
    df['feature_18+feature_23+feature_3*feature_21'] = df['feature_18'] + df['feature_23'] + df['feature_3'] * df['feature_21']
    df['feature_18+feature_23+feature_3+feature_21'] = df['feature_18'] + df['feature_23'] + df['feature_3'] + df['feature_21']
    df['feature_23+feature_8+feature_6+feature_21'] = df['feature_23'] + df['feature_8'] + df['feature_6'] + df['feature_21']
    df['feature_17+feature_23+feature_16+feature_21'] = df['feature_17'] + df['feature_23'] + df['feature_16'] + df['feature_21']
    df['feature_23+feature_3+feature_8*feature_21'] = df['feature_23'] + df['feature_3'] + df['feature_8'] * df['feature_21']
    df['feature_23+feature_8+feature_21+feature_13'] = df['feature_23'] + df['feature_8'] + df['feature_21'] + df['feature_13']
    df['feature_17+feature_23+feature_11*feature_21'] = df['feature_17'] + df['feature_23'] + df['feature_11'] * df['feature_21']
    df['feature_23+feature_8+feature_6*feature_21'] = df['feature_23'] + df['feature_8'] + df['feature_6'] * df['feature_21']
    df['feature_18+feature_23+feature_16+feature_21'] = df['feature_18'] + df['feature_23'] + df['feature_16'] + df['feature_21']
    df['feature_19+feature_23+feature_8+feature_21'] = df['feature_19'] + df['feature_23'] + df['feature_8'] + df['feature_21']
    df['feature_23+feature_8+feature_21*feature_13'] = df['feature_23'] + df['feature_8'] + df['feature_21'] * df['feature_13']
    df['feature_17+feature_8+feature_6+feature_21'] = df['feature_17'] + df['feature_8'] + df['feature_6'] + df['feature_21']
    df['feature_17+feature_23+feature_11+feature_21'] = df['feature_17'] + df['feature_23'] + df['feature_11'] + df['feature_21']
    df['feature_18+feature_23+feature_11*feature_21'] = df['feature_18'] + df['feature_23'] + df['feature_11'] * df['feature_21']
    df['feature_23+feature_8+feature_2*feature_21'] = df['feature_23'] + df['feature_8'] + df['feature_2'] * df['feature_21']
    df['feature_23+feature_8+feature_2+feature_21'] = df['feature_23'] + df['feature_8'] + df['feature_2'] + df['feature_21']
    df['feature_18+feature_8+feature_6+feature_21'] = df['feature_18'] + df['feature_8'] + df['feature_6'] + df['feature_21']
    df['feature_18+feature_23+feature_11+feature_21'] = df['feature_18'] + df['feature_23'] + df['feature_11'] + df['feature_21']
    df['feature_17+feature_23+feature_4*feature_21'] = df['feature_17'] + df['feature_23'] + df['feature_4'] * df['feature_21']
    df['feature_23+feature_11+feature_8*feature_21'] = df['feature_23'] + df['feature_11'] + df['feature_8'] * df['feature_21']
    df['feature_17+feature_5+feature_8+feature_21'] = df['feature_17'] + df['feature_5'] + df['feature_8'] + df['feature_21']
    df['feature_23+feature_22+feature_8+feature_21'] = df['feature_23'] + df['feature_22'] + df['feature_8'] + df['feature_21']
    df['feature_17+feature_23+feature_4+feature_21'] = df['feature_17'] + df['feature_23'] + df['feature_4'] + df['feature_21']
    df['feature_18+feature_23+feature_4*feature_21'] = df['feature_18'] + df['feature_23'] + df['feature_4'] * df['feature_21']
    df['feature_19+feature_23+feature_3+feature_21'] = df['feature_19'] + df['feature_23'] + df['feature_3'] + df['feature_21']
    df['feature_17+feature_19+feature_23*feature_21'] = df['feature_17'] + df['feature_19'] + df['feature_23'] * df['feature_21']
    df['feature_18+feature_5+feature_8+feature_21'] = df['feature_18'] + df['feature_5'] + df['feature_8'] + df['feature_21']
    df['feature_19+feature_23+feature_16*feature_21'] = df['feature_19'] + df['feature_23'] + df['feature_16'] * df['feature_21']
    df['feature_18+feature_19+feature_23*feature_21'] = df['feature_18'] + df['feature_19'] + df['feature_23'] * df['feature_21']
    df['feature_17+feature_23+feature_2*feature_21'] = df['feature_17'] + df['feature_23'] + df['feature_2'] * df['feature_21']
    df['feature_23+feature_4+feature_8*feature_21'] = df['feature_23'] + df['feature_4'] + df['feature_8'] * df['feature_21']
    df['feature_18+feature_23+feature_4+feature_21'] = df['feature_18'] + df['feature_23'] + df['feature_4'] + df['feature_21']
    df['feature_23+feature_8+feature_10+feature_21'] = df['feature_23'] + df['feature_8'] + df['feature_10'] + df['feature_21']
    df['feature_19+feature_23+feature_16+feature_21'] = df['feature_19'] + df['feature_23'] + df['feature_16'] + df['feature_21']
    df['feature_19+feature_8+feature_6+feature_21'] = df['feature_19'] + df['feature_8'] + df['feature_6'] + df['feature_21']
    df['feature_17+feature_18+feature_23*feature_21'] = df['feature_17'] + df['feature_18'] + df['feature_23'] * df['feature_21']
    df['feature_23+feature_8+feature_10*feature_21'] = df['feature_23'] + df['feature_8'] + df['feature_10'] * df['feature_21']
    return df

In [5]:
train = additional_features(train)

In [6]:
features = [f for f in train.columns if 'feature' in f]

In [8]:
def scorer(y_test, y_pred):
    score = (stats.spearmanr(y_test, y_pred))[0]
    print('Score as calculated for the leader board (っಠ‿ಠ)っ {}'.format(score))
    return score

In [66]:
X_train, X_test, y_train, y_test = train_test_split(train[features], target.target_b, test_size=0.25, shuffle=False)
import lightgbm as lgb
params = {
            'boosting_type': 'gbdt',
            'objective': 'regression',
            'metric': 'rmse',
            'subsample': 0.75,
            'subsample_freq': 1,
            'learning_rate': 0.01,
            'feature_fraction': 0.9,
            'max_depth': 4,
            'lambda_l1': 1,  
            'lambda_l2': 1,
            }
def scorer(y_test, y_pred):
    score = (stats.spearmanr(y_test, y_pred))[0]
    print('Score as calculated for the leader board (っಠ‿ಠ)っ {}'.format(score))
    # return score
model_one_feature = lgb.LGBMRegressor(**params, random_state=2021)
model_one_feature.fit(X_train, y_train, eval_set=[(X_test, y_test)], early_stopping_rounds=100, verbose=100)
pred = model_one_feature.predict(X_test)
scorer(y_test, pred)

Training until validation scores don't improve for 100 rounds.
[100]	valid_0's rmse: 0.279056
Did not meet early stopping. Best iteration is:
[100]	valid_0's rmse: 0.279056
Score as calculated for the leader board (っಠ‿ಠ)っ 0.6721652827172998


In [67]:
lgbm_features = pd.Series(model_one_feature.booster_.feature_importance(importance_type='gain'), index=features).sort_values(ascending=False)

In [68]:
lgbm_features_list = list(lgbm_features.index)

In [69]:
lgbm_features_list[0:1]

['feature_17+feature_23+feature_11*feature_21']

In [70]:
X_train, X_test, y_train, y_test = train_test_split(train[lgbm_features_list[0:1]], target.target_b, test_size=0.25, shuffle=False)
model_one_feature = lgb.LGBMRegressor(**params, random_state=2021)
model_one_feature.fit(X_train, y_train, eval_set=[(X_test, y_test)], early_stopping_rounds=100, verbose=100)
pred = model_one_feature.predict(X_test)
scorer(y_test, pred)

Training until validation scores don't improve for 100 rounds.
[100]	valid_0's rmse: 0.280682
Did not meet early stopping. Best iteration is:
[100]	valid_0's rmse: 0.280682
Score as calculated for the leader board (っಠ‿ಠ)っ 0.6692334142480808


In [71]:
sp_score = 0.6692334142480808

In [72]:
def scorer(y_test, y_pred):
    score = (stats.spearmanr(y_test, y_pred))[0]
    # print('Score as calculated for the leader board (っಠ‿ಠ)っ {}'.format(score))
    return score

In [73]:
tol = 0.0001

features_to_keep= lgbm_features_list[0:1]
count = 2
for f in lgbm_features_list[1:]:
    print()
    print('testing feature: ', f, ' which is feature ', count,' out of ', len(lgbm_features_list))
    count = count + 1
    X_train, X_test, y_train, y_test = train_test_split(train[features_to_keep + [f]], target.target_b, test_size=0.25, shuffle=False)
    
    model_ = lgb.LGBMRegressor(**params, random_state=2021)
    model_.fit(X_train, y_train, eval_set=[(X_test, y_test)], early_stopping_rounds=100, verbose=100)
    pred = model_.predict(X_test)
    sp_score_cons = scorer(y_test, pred)

    print('New Test Spearman score ={}'.format((sp_score_cons)))
    print('Previous features Spearman score ={}'.format((sp_score)))

    diff_sp = sp_score_cons - sp_score
    if diff_sp >= tol:
        print('Increase in SP SCORE ={}'.format(diff_sp))
        print('keep: ', f)

        sp_score = sp_score_cons
        features_to_keep.append(f)

    else:
        print('Increase in SP SCORE ={}'.format(diff_sp))
        print('remove: ', f)
        print()
    print('Features list till now: ' , len(features_to_keep))


testing feature:  feature_17+feature_23+feature_11+feature_21  which is feature  2  out of  84
Training until validation scores don't improve for 100 rounds.
[100]	valid_0's rmse: 0.281123
Did not meet early stopping. Best iteration is:
[100]	valid_0's rmse: 0.281123
New Test Spearman score =0.6686600449382724
Previous features Spearman score =0.6692334142480808
Increase in SP SCORE =-0.0005733693098084469
remove:  feature_17+feature_23+feature_11+feature_21

Features list till now:  1

testing feature:  feature_23+feature_11+feature_8*feature_21  which is feature  3  out of  84
Training until validation scores don't improve for 100 rounds.
[100]	valid_0's rmse: 0.281341
Did not meet early stopping. Best iteration is:
[100]	valid_0's rmse: 0.281341
New Test Spearman score =0.6663638131619358
Previous features Spearman score =0.6692334142480808
Increase in SP SCORE =-0.0028696010861449794
remove:  feature_23+feature_11+feature_8*feature_21

Features list till now:  1

testing feature: 

In [74]:
features_to_keep

['feature_17+feature_23+feature_11*feature_21',
 'feature_17+feature_23+feature_4*feature_21',
 'feature_18+feature_23+feature_4*feature_21',
 'feature_18+feature_23+feature_16*feature_21',
 'feature_18+feature_23+feature_3*feature_21',
 'feature_28',
 'feature_29',
 'feature_25',
 'feature_17',
 'feature_23+feature_22+feature_8+feature_21',
 'feature_10',
 'feature_27']

In [64]:
features_r = ['feature_17+feature_23+feature_8*feature_21',
 'feature_18+feature_23+feature_8*feature_21',
 'feature_17+feature_23+feature_8+feature_21',
 'feature_29',
 'feature_19+feature_23+feature_8*feature_21',
 'feature_18+feature_23+feature_8+feature_21',
 'feature_10',
 'feature_6']

In [77]:
features_g = ['feature_23+feature_11+feature_8+feature_21',
 'feature_23+feature_4+feature_8+feature_21',
 'feature_17+feature_23+feature_8*feature_21',
 'feature_23+feature_3+feature_8+feature_21',
 'feature_17+feature_23+feature_16*feature_21',
 'feature_28',
 'feature_29',
 'feature_25',
 'feature_18+feature_23+feature_8*feature_21',
 'feature_18+feature_23+feature_16+feature_21',
 'feature_26',
 'feature_17+feature_23+feature_3*feature_21',
 'feature_17+feature_18+feature_23*feature_21',
 'feature_23+feature_8+feature_10*feature_21',
 'feature_4',
 'feature_10',
 'feature_19+feature_23+feature_16*feature_21',
 'feature_5',
 'feature_8']

In [78]:
features_b = ['feature_17+feature_23+feature_11*feature_21',
 'feature_17+feature_23+feature_4*feature_21',
 'feature_18+feature_23+feature_4*feature_21',
 'feature_18+feature_23+feature_16*feature_21',
 'feature_18+feature_23+feature_3*feature_21',
 'feature_28',
 'feature_29',
 'feature_25',
 'feature_17',
 'feature_23+feature_22+feature_8+feature_21',
 'feature_10',
 'feature_27']

In [80]:
import optuna
X_train, X_test, y_train, y_test = train_test_split(train[features_g], target.target_g, test_size=0.25, shuffle=False)

def scorer(y_test, y_pred):
    score = (stats.spearmanr(y_test, y_pred))[0]
    print('Score as calculated for the leader board (っಠ‿ಠ)っ {}'.format(score))
    return score

N_TRIALS = 1000
def objective(trial):    
    params = {
            'boosting_type': 'gbdt',
            'objective': 'regression',
            'metric': 'rmse',
            'num_leaves': trial.suggest_int('num_leaves', 4, 32),
            'max_depth': trial.suggest_int('max_depth', 4, 16),
            'learning_rate' : trial.suggest_uniform('learning_rate', 0.0001, 0.99),
            'n_estimators': trial.suggest_int('n_estimators', 100, 5000),
            'min_split_gain' : trial.suggest_uniform('min_split_gain', 0, 1),
            'min_child_weight': trial.suggest_uniform('min_child_weight', 1e-3, 1),
            'min_child_samples': trial.suggest_int('min_child_samples',20, 500),
            'subsample' : trial.suggest_uniform('subsample', 0.5, 1),
            'subsample_freq': trial.suggest_int('subsample_freq', 0, 1),
            'reg_alpha': trial.suggest_loguniform('reg_alpha', 1e-8, 1.0),
            'reg_lambda': trial.suggest_loguniform('reg_lambda', 1e-8, 1.0),

            }

    model = lgb.LGBMRegressor(**params, random_state=2021)
    # wrapper = RegressorChain(model, [0, 1, 2], random_state=2021)
    # wrapper.fit(X_train, y_train)
    # preds = wrapper.predict(X_test)
    model.fit(X_train, y_train, eval_set=[(X_test, y_test)], early_stopping_rounds=100, verbose=100)
    preds = model.predict(X_test)
    
    
    score = scorer(y_test, preds)
    
    from google.colab import output
    output.clear()
    return score
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=N_TRIALS, show_progress_bar= True)
display(study.best_params)
display(study.best_trial.value)

[I 2021-03-14 08:13:27,179] Trial 340 finished with value: 0.541219083180964 and parameters: {'num_leaves': 32, 'max_depth': 15, 'learning_rate': 0.0004900820824294886, 'n_estimators': 4982, 'min_split_gain': 0.9784852734402016, 'min_child_weight': 0.3654650150961478, 'min_child_samples': 453, 'subsample': 0.5435998642515718, 'subsample_freq': 1, 'reg_alpha': 0.0001358772967592931, 'reg_lambda': 1.3342132380913252e-05}. Best is trial 289 with value: 0.5412934998096476.
Training until validation scores don't improve for 100 rounds.
[100]	valid_0's rmse: 0.344976
[200]	valid_0's rmse: 0.338404
[300]	valid_0's rmse: 0.332713
[400]	valid_0's rmse: 0.327795
[500]	valid_0's rmse: 0.323557



KeyboardInterrupt: ignored

In [ ]:
{'num_leaves': 32, 
 'max_depth': 15,
 'learning_rate': 0.0004900820824294886, 
 'n_estimators': 4982,
 'min_split_gain': 0.9784852734402016, 
 'min_child_weight': 0.3654650150961478,
 'min_child_samples': 453, 
 'subsample': 0.5435998642515718, 
 'subsample_freq': 1, 
 'reg_alpha': 0.0001358772967592931, 
 'reg_lambda': 1.3342132380913252e-05}